# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadithah,34.1397,42.3781,27.07,26,100,3.20,IQ,1681932547
1,1,olonkinbyen,70.9221,-8.7187,1.90,90,3,7.54,SJ,1681932403
2,2,waitangi,-43.9535,-176.5597,16.57,93,100,3.58,NZ,1681932410
3,3,ushuaia,-54.8000,-68.3000,9.79,81,75,12.35,AR,1681932312
4,4,tura,25.5198,90.2201,26.66,61,51,4.12,IN,1681932466


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
vacation_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scale=2,
    color="City"
)
# Display the map
vacation_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
perfect_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 20) & (city_data_df["Humidity"] < 70),:]

# Drop any rows with null values
perfect_weather_df = perfect_weather_df.dropna()

# Display sample data
perfect_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,taksimo,56.3444,114.8794,-7.00,69,100,1.91,RU,1681932550
12,12,laguna,38.4210,-121.4238,17.38,47,0,3.60,US,1681932550
19,19,thompson,55.7435,-97.8558,-0.91,51,75,6.17,CA,1681932406
20,20,mar del plata,-38.0023,-57.5575,15.53,62,40,7.20,AR,1681932553
27,27,badger,64.8000,-147.5333,-0.72,52,40,3.09,US,1681932404


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = perfect_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,taksimo,56.3444,114.8794,-7.00,69,100,1.91,RU,1681932550,
12,12,laguna,38.4210,-121.4238,17.38,47,0,3.60,US,1681932550,
19,19,thompson,55.7435,-97.8558,-0.91,51,75,6.17,CA,1681932406,
20,20,mar del plata,-38.0023,-57.5575,15.53,62,40,7.20,AR,1681932553,
27,27,badger,64.8000,-147.5333,-0.72,52,40,3.09,US,1681932404,
...,...,...,...,...,...,...,...,...,...,...,...
542,542,sabzevar,36.2126,57.6819,15.75,15,0,4.25,IR,1681932636,
544,544,sarmiento,-45.5882,-69.0700,19.54,49,100,16.53,AR,1681932637,
552,552,mirnyy,62.5353,113.9611,-5.98,48,100,5.81,RU,1681932637,
553,553,kittila,67.6647,24.8936,7.08,65,0,5.14,FI,1681932460,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"
params = {"apiKey": geoapify_key,
          "categories":categories,
          "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taksimo - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
thompson - nearest hotel: Thompson Inn
mar del plata - nearest hotel: Nuevo Ostende
badger - nearest hotel: Hotel North Pole
bredasdorp - nearest hotel: Victoria Hotel
jamestown - nearest hotel: DoubleTree Jamestown
bethel - nearest hotel: Hampton Inn Danbury
longyearbyen - nearest hotel: The Vault
yellowknife - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
kruisfontein - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
puerto natales - nearest hotel: Dorotea Patagonia Hostel
floro - nearest hotel: Comfort Hotel Victoria Florø
albany - nearest hotel: No hotel found
qarazhal - nearest hotel: No hotel found
rumyantsevo - nearest hotel: No hotel found
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
vrede - nearest hotel: No hotel found
kungur -

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,taksimo,56.3444,114.8794,-7.00,69,100,1.91,RU,1681932550,No hotel found
12,12,laguna,38.4210,-121.4238,17.38,47,0,3.60,US,1681932550,Holiday Inn Express & Suites
19,19,thompson,55.7435,-97.8558,-0.91,51,75,6.17,CA,1681932406,Thompson Inn
20,20,mar del plata,-38.0023,-57.5575,15.53,62,40,7.20,AR,1681932553,Nuevo Ostende
27,27,badger,64.8000,-147.5333,-0.72,52,40,3.09,US,1681932404,Hotel North Pole
...,...,...,...,...,...,...,...,...,...,...,...
542,542,sabzevar,36.2126,57.6819,15.75,15,0,4.25,IR,1681932636,مامورسرا مخابرات سبزوار
544,544,sarmiento,-45.5882,-69.0700,19.54,49,100,16.53,AR,1681932637,Ismar
552,552,mirnyy,62.5353,113.9611,-5.98,48,100,5.81,RU,1681932637,Зарница
553,553,kittila,67.6647,24.8936,7.08,65,0,5.14,FI,1681932460,Hotelli Kittilä


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name"]
)

# Display the map plot
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)